# Assignment 03
## Web Search
## CSCI S-96    

> **Instructions:** For this assignment you will complete the exercises shown. All exercises involve creating and executing some Python code. Additionally, most exercises have questions for you to answer. You can answer questions by creating a Markdown cell and writing your answer. If you are not familiar with Markdown, you can find a brief tutorial [here](https://www.markdownguide.org/cheat-sheet/).     

In this assignment you will gain some experience and insight into how web search algorithms work. Specifically, you will implement versions of three algorithms, simple PageRank, PageRank with damping, and the HITS algorithm. All three of these algorithms use a directed graph model of the web.   

Data small scale examples and coding methods used here are not directly scalable to web sized problems. Rather, the point is for you to understand the basic characteristics of these web search algorithms. Web scale searching requires massive resources not readily available to most people. 

## Simple PageRank Example

To get a feeling for the basics of the PageRank algorithm you will create and test simple code. 

As a first step, execute the code in the cell below to import the packages you will need for the exercises. 

In [1]:
import numpy as np

We will start with a simple example. Figure 1 shows a set of web pages and their hyperlinks. This is a directed graph with the pages as nodes and the hyperlinks as the directed edges. This graph is **complete**. Every page is accessable from any other page, possibly with visits to intermediate nodes required.  

<img src="../images/web1.png" alt="Drawing" style="width:500px; height:400px"/>
<center>Figure 1: A small set of web pages</center>

The directed edges of the graph define the association between the nodes. A directed edge, or hyperlink, runs from a node's column to another node's row. The association is binary. The directed edge either exists or it does not. 

> **Exercise 03-1:** In the cell below you will create the association matrix and the initial page probability vector. Do the following:  
> 1. Create the association matrix, $A$, using [numpy.array](https://numpy.org/doc/stable/reference/generated/numpy.array.html). This matrix is constructed with a 1 where a page in a column links to another page, and 0 elsewhere.  
> 2. Print the shape of your association matrix as a check of your association matrix.
> 3. Print the in degree and out degree of your association matrix, using [numpy.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html). Set the argument `axis` to 1 to sum across rows and 0 to sum down columns. 
> 3. Create the uniformly distributed page probability vector, $p$, using `numpy.array`. 
> 4. Verify that the page probability vector sums to 1.0 using `numpy.sum`.

In [2]:
## Create the Association Matrix and print the summary
## Put your code below
A = np.array([[0,1,0,1,0],
            [1,0,0,0,0],
            [1,0,0,0,0],
            [1,1,1,0,1],
            [0,1,1,1,0]])
print(A.shape)
print(np.sum(A, axis=1))
print(np.sum(A, axis=0))

## Create the equal probability starting values
## Put your code below
p = np.array([0.2,0.2,0.2,0.2,0.2])
print(np.sum(p))

(5, 5)
[2 1 1 4 3]
[3 3 2 2 1]
1.0


> Answer the following questions:  
> - Are the out degree and in degree you computed from the association matrix consistent with the graph? 
> - Are the page probabilities a proper distribution summing to 1.0? 
> **End of exercise.**

For the PageRank algorithm we must normalize the association matrix by the inverse of the out degree of each page. This normalization uniformly distributes the influence of each page on the PageRank of the other pages over its out degree. The calculation of the out degree of a network is illustrated in Figure 2.  

<img src="../images/outdegree.png" alt="Drawing" style="width:400px; height:200px"/>
<center>Figure 2: Example of computing out degree from an association matrix</center>

The normalized transition probability matrix, $M$, is then computed from the association matrix, $A$: 

$$M = A D^{-1}$$

Where, $D^{-1}$ is the inverse of a matrix with the out degree values on the diagonal and zeros elsewhere.  

You can see from the foregoing that $M$ distributes the influence of the page by the in|verse of the out degree. In other words, the influence is inversely weighted by the number of pages each page links to. 

> **Exercise 03-2:** You will now compute the normalized transition matrix, $M$. To do so create a function called `norm_association` with the association matrix as the argument. Do the following: 
> 1. Create your function `norm_association` which will do the following:  
>    - Compute the sum of the columns of the association matrix using `numpy.sum` with the `axis=0` argument to sum along columns. 
>   - Compute the inverse of the column sums as a vector. Be sure to avoid zero divides, which will occur in subsequent exercises. If the column sum is 0 set the inverse to zero.   
>   - Create a square diagonal matrix from the inverse column sums using [numpy.diag](https://numpy.org/doc/stable/reference/generated/numpy.diag.html) to form the inverse out degree diagonal matrix.
>  - Finally, return the matrix product of the association matrix and inverse out degree matrix using [numpy.matmul](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html).  
2. Execute your function on the association matrix you computed. Save and print the normalized transition matrix, and examine the result.  

In [3]:
def norm_association(A):
    '''Function to normalize the association matrix by out degree.
    The function accounts for cases where the column sum is 0'''
    ## Put your code below
    col_sum = np.sum(A, axis=0)
    d = [0.0]*len(col_sum)
    for i,x in enumerate(col_sum): 
        if x > .0001: d[i] = 1.0/x  
    D_inv = np.diag(d)
    return np.matmul(A,D_inv)

## Execute the function
## Put your code below
M = norm_association(A)
M

array([[0.        , 0.33333333, 0.        , 0.5       , 0.        ],
       [0.33333333, 0.        , 0.        , 0.        , 0.        ],
       [0.33333333, 0.        , 0.        , 0.        , 0.        ],
       [0.33333333, 0.33333333, 0.5       , 0.        , 1.        ],
       [0.        , 0.33333333, 0.5       , 0.5       , 0.        ]])

> Does the result look correct based on the out degree of each page on the graph?  
> **End of exercise.**

With the transition probability matrix, $M$ computed it is time to investigate the convergence of the PageRank algorithm. You can think of the PageRank algorithm as a series of transitions of a Markov Chain. Given the transition probability matrix, $M$, the update, or single Markov transition, of the page probabilities, $p_i$, is computed: 

$$p_i = M p_{i-1}$$

The Markov chain can be executed for a great many transitions. The result of $n$ transitions, starting from an initial set of page probabilities, $p_0$, can be written:  

$$p_n = M^n p_{0}$$

At convergence the page probabilities, $p_n$, approach a constant or **steady state** value. This steady state probability vector values are the PageRank of the web pages.  

> **Exercise 03-3:** You will now create and execute code with the goal of getting a feel for how the page probabilities change for a single transition of a Markov process. The accomplish this task you will create a function called `transition` with arguments of the the normalized transition probability matrix and the vector of page probabilities. Specifically you will:   
> 1. Create the function `transition` which uses [numpy.dot](https://numpy.org/doc/stable/reference/generated/numpy.dot.html) to compute the product of the transition matrix and the page probability vector.  
> 2. Execute the `transition` function on the normalized transition probability matrix and vector of initial page probabilities you have created, saving the result. 
> 3. Print the Euclidean (L2) norm of the difference between the initial page probabilities and the updated page probabilities. 
> 4. Print the sum of the page probabilities computed with `transition` along with the actual vector of values. 

In [4]:
def transition(transition_probs, probs):
    '''Function to compute the probabilities resulting from a 
    single transition of a Markov process'''
    ## Put your code below
    return np.dot(transition_probs, probs)

## Compute probabilities after first state trasnition and print the sumamries
## Put your code below
p1 = transition(M, p)
print(np.linalg.norm(np.subtract(p,p1)))
print(np.sum(p1))
p1

0.30912061651652345
1.0


array([0.16666667, 0.06666667, 0.06666667, 0.43333333, 0.26666667])

> Is the sum of the page probabilities equal to 1.0 as it should be? Considering in degree of the pages, are the relative changes in the page probabilities what you would expect and why.  
> **End of exercise.**

> **Exercise 03-4:** You will continue with computing transitions of the Markov chain. Use the `transition` function with the normalized transition probability matrix and the page probability vector computed from the first transition as arguments. Your code must do the following:  
> 1. Compute the resulting page probabilities of the second transition.
> 2. Compute and print the Euclidean (L2) norm of the difference between the page probabilities before and after the transition. 
> 3. Compute and print the sum of the page probabilities. 
> 4. Display the page probabilities. 

In [5]:
# Compute probabilities after second state transition
## Put your code below
p2 = transition(M, p1)
print(np.linalg.norm(np.subtract(p2,p1)))
print(np.sum(p2))
p2

0.09262962222518371
1.0


array([0.23888889, 0.05555556, 0.05555556, 0.37777778, 0.27222222])

> Note the difference between the Euclidean norms of the differences for the first and second transition calculations. Does the change in this difference from one step to the next indicate the algorithm is converging to the steady state probabilities?  
> **End of exercise.**

> **Exercise 03-4:** The question now is how does this simplified version of page rank converge with more iterations? To find out, do the following:   
> 1. Create a function `pagerank1` with arguments, the normalized transition matrix, the initial page probabilities and a convergence threshold value of 0.01, which does the following:  
>    - Initialize a euclidean distance norm to 1.0 and the resulting page probabilities to a vector of 0.0 values of length equal to the dimension of the transition matrix.   
>    - Set a loop counter to 1.  
>    - Use a 'while' loop with termination conditions the euclidean distance norm less than the threshold value AND the loop counter less than 50.  In side this loop do the following:  
>      1. Update the page probabilities using the `transition` function you created. 
>      2. Compute the Euclidean norm of the difference between the initial and updated page probabilities from the transition.   
>      3. Print the value of the loop counter and the Euclidean norm value. 
>      4. Copy the updated page probability vector into the input page probability vector.  
>      5. Increment the loop counter by 1. 
>    - Return the page probabilities at convergence.  
> 2. Execute your `pagerank1` function using the transition matrix and initial page probability vector. 

In [6]:
# Compute probabilities after a larger number of state transitions 
def pagerank1(M, in_probs,  threshold = 0.01):  
    euclidean_dist = 1.0 
    page_probabilities = np.array([0.0]*len(M))
    i = 1   
    ## Put your code below
    while euclidean_dist > threshold and i < 50:    
        page_probabilities = transition(M, in_probs)
        euclidean_dist = np.linalg.norm(np.subtract(in_probs, page_probabilities)) 
        in_probs =  page_probabilities
        print('Iteration ' + str(i) + ' Euclidean distance = ' + str(euclidean_dist))
        i += 1
    return page_probabilities    

## Execute your function
## Put your code below
pagerank1(M,p)

Iteration 1 Euclidean distance = 0.30912061651652345
Iteration 2 Euclidean distance = 0.09262962222518371
Iteration 3 Euclidean distance = 0.0627447198003608
Iteration 4 Euclidean distance = 0.04713034716117629
Iteration 5 Euclidean distance = 0.04044983066499079
Iteration 6 Euclidean distance = 0.034751469502374295
Iteration 7 Euclidean distance = 0.02965169022900917
Iteration 8 Euclidean distance = 0.025297173726972686
Iteration 9 Euclidean distance = 0.021602992445622173
Iteration 10 Euclidean distance = 0.018452365792341184
Iteration 11 Euclidean distance = 0.01575978617875431
Iteration 12 Euclidean distance = 0.013459509556315313
Iteration 13 Euclidean distance = 0.011495031885715244
Iteration 14 Euclidean distance = 0.00981734526591825


array([0.21890828, 0.07149265, 0.07149265, 0.38258726, 0.25551917])

> Answer the following questions:  
> - Does the convergence of this algorithm seem fairly rapid? 
> - Does the rank order of the computed page probabilities make sense given the directed graph of the pages? 
> **End of exercise.**

## A More Complicated Example   

You will now work with a more complicated example The graph of 6 web pages, shown in Figure 2, is no longer complete. The out degree of page 6 is 0. A random surfer transitioning to page 6 will have no escape, a **spider trap**! 

<img src="../images/web2.png" alt="Drawing" style="width:500px; height:500px"/>
<center>Figure 3: A small set of web pages with a dead end</center>

> **Exercise 03-5:** You will now create both the normalized transition matrix and the initial page probability vector for the graph of Figure 3. In this exercise you will . Do the following:  
> 1. Create the association matrix and normalize it using your `norm_association` function. Name your transition matrix M_deadend. Print the result. 
> 2. Create a vector of containing the the in degree of the graph. Normalize the vector so the initial probabilities sum to 1.0. Save and print the result.   

In [7]:
## Create the Association Matrix
## Put your code below
A_deadend = np.array([[0,1,0,1,0,0],
                      [1,0,0,0,0,0],
                      [1,0,0,0,0,0],
                      [1,1,1,0,1,0],
                      [0,1,1,1,0,0],
                     [1,0,1,0,0,0]])

## Normalize the association matrix
## Put your code below
M_deadend = norm_association(A_deadend)
print(M_deadend)

## Create the equal probability starting values
## Put your code below
p_deadend = np.sum(A_deadend, axis=1)
p_deadend = np.divide(p_deadend, np.sum(p_deadend))
print(p_deadend)

[[0.         0.33333333 0.         0.5        0.         0.        ]
 [0.25       0.         0.         0.         0.         0.        ]
 [0.25       0.         0.         0.         0.         0.        ]
 [0.25       0.33333333 0.33333333 0.         1.         0.        ]
 [0.         0.33333333 0.33333333 0.5        0.         0.        ]
 [0.25       0.         0.33333333 0.         0.         0.        ]]
[0.15384615 0.07692308 0.07692308 0.30769231 0.23076923 0.15384615]


> Examine your results. Are the 0 values for the transition probabilities of page 6 and the associated non-zero in the page probabilities consistent with the graph of these pages?    
> **End of exercise.**   

> **Exercises 03-6:** What happens if you apply the simplified PageRank algorithm to the pages on a graph that is not complete, like the one shown in Figure 2? To find out, execute your `pagerank1` function with arguments `M_deadend`, `p_deadend` and `threshold=0.001`. The smaller threshold value is to ensure convergence. 

In [8]:
## Put your code below
pagerank1(M_deadend,p_deadend, threshold = 0.001)

Iteration 1 Euclidean distance = 0.11176663957796601
Iteration 2 Euclidean distance = 0.05047440944879368
Iteration 3 Euclidean distance = 0.03196212671293077
Iteration 4 Euclidean distance = 0.02896215595961246
Iteration 5 Euclidean distance = 0.02609541048864403
Iteration 6 Euclidean distance = 0.024648655232248615
Iteration 7 Euclidean distance = 0.021962192704449258
Iteration 8 Euclidean distance = 0.020672129115029844
Iteration 9 Euclidean distance = 0.01857250748188292
Iteration 10 Euclidean distance = 0.01740991694999164
Iteration 11 Euclidean distance = 0.015726745028649337
Iteration 12 Euclidean distance = 0.014693350586384997
Iteration 13 Euclidean distance = 0.013329322358288835
Iteration 14 Euclidean distance = 0.01241850218375155
Iteration 15 Euclidean distance = 0.011302786548359096
Iteration 16 Euclidean distance = 0.010506080760252699
Iteration 17 Euclidean distance = 0.0095866222704363
Iteration 18 Euclidean distance = 0.008894086069700165
Iteration 19 Euclidean distan

array([0.00485108, 0.00131723, 0.00131723, 0.00805744, 0.0053278 ,
       0.00179396])

> Answer the following questions:  
> - Given the lower convergence threshold (factor of 10) does the convergence of the algorithm generally seem fast? 
> - Examine the page probabilities, noticing that the sum is far less than 1.0. Does this indicate that the algorithm is converging to a point where all page probabilities are 0.0?  
> **End of exercise.**

It is clear from the results of the foregoing exercise that the simple PageRank algorithm does not converge to a usable set of page probabilities when faced with graph that is not complete.Fortunately, there is a simple fix, add a damping term. You can think of the damping term as allowing a random surfer to make an arbitrary transition or jump with some small probability. These random jumps help the random surfer to better explore the graph and to escape from spider traps. The jump probabilities from states, $p_i$, are a function of the damping factor $d$: 

$$Jump\ Probability = \frac{(1-d)}{n}$$

Where $n$ is the dimension of the transition probability matrix. 

The updated page probabilities, $p_i$, are then computed with the damped PageRank algorithm as:   

$$p_{i} = d * M p_{i-1} + \frac{(1-d)}{n}$$

Where $M$ is the transition probability matrix and p are the initial page probability values. 

> **Exercise 03-7:** To implement the PageRank algorithm with a damping factor do the following:  
> 1. Create a `transition_damped` function with arguments, the transition probability matrix, the initial page probabilities, and the damping factor, $d=0.85$, which does the following:  
>   - Compute the updated page probabilities by multiplying the inner (dot) product of the transition probability matrix by the initial page probabilities by the damping factor, `d`. Use the [numpy.multiply](https://numpy.org/doc/stable/reference/generated/numpy.multiply.html) and numpy.dot functions.
>   - Compute the jump probabilities vector of length the dimension of the transition matrix and multiply these by element wise by the page probabilities using `np.multiply`. *Note:* the jump probabilities are constant, so you can create code that only computes them once if you so choose.  
>   - Return the sum of the damped page probabilities and the jump probabilities.  
> 2. Create a `pagerank_damped` function. This function is identical to the `pagerank1` function you already created except that it uses the `transiton_damped` function in place of the `transition` function.  
> 3. Call your `pagerank_damped` function using arguments of `M_deadend` and `p_deadend`. 

In [11]:
## Add a damping facgtor to the transiton 
def transition_damped(transition_probs, probs, d=0.85):
    '''Function to compute the probabilities resulting from a 
    single transition of a Markov process including a damping
    factor to deal with dead ends'''
    ## Put your code below
    out = np.multiply(np.dot(transition_probs, probs), d)
    damping = np.array([(1-d)/len(probs)]*len(probs))
    out = np.add(out,damping)
    return out

## function for the PageRank algorithm using the damped transition algorithm 
## Put your code below
def pagerank_damped(M, in_probs,  threshold = 0.01):  
    euclidean_dist = 1.0 
    page_probabilities = np.array([0.0]*len(M))
    i = 1
    while euclidean_dist > threshold and i < 50:    
        page_probabilities = transition_damped(M, in_probs)
        euclidean_dist = np.linalg.norm(np.subtract(in_probs, page_probabilities)) 
        in_probs =  page_probabilities
        print('Iteration ' + str(i) + ' Euclidean distance = ' + str(euclidean_dist))
        i += 1
    return page_probabilities    

## Execute your funciton
## Put your code below
damped_rank = pagerank_damped(M_deadend,p_deadend)
print(sum(damped_rank))
damped_rank

Iteration 1 Euclidean distance = 0.08901259061855184
Iteration 2 Euclidean distance = 0.037944951600823416
Iteration 3 Euclidean distance = 0.02079410296917142
Iteration 4 Euclidean distance = 0.016465473690449515
Iteration 5 Euclidean distance = 0.012780435459627378
Iteration 6 Euclidean distance = 0.010158493874458772
Iteration 7 Euclidean distance = 0.007796135866124159
0.6847979085593676


array([0.13562668, 0.05462989, 0.05462989, 0.21823996, 0.15133414,
       0.07033734])

> Answer the following questions:   
> - Notice the convergence of the damped page rank algorithm. Compare this convergence to that of the undamped PageRank algorithm on the complete graph of 5 pages. Do you think this change is a result of the jumps the random surfer makes?   
> - Examine the final page probabilities. Does the rank of these page probabilities make sense given the graph of the pages in this case? 
> **End of exercise.**

## Hubs, Authorities, and the HITS Algorithm  

The hubs and authorities model is an alternative to PageRank. Rather than using a single metric to rank the importance of web pages, the **HITS** algorithm iteratively updates the hub and authority scores for each of the pages. 

The HITS algorithm updates the authority and hub scores iteratively. The authority score is sum of the hubs linked to it: 
$$𝑎= \beta 𝐴 ℎ$$

Hub score is sum of the authorities it links to: 
$$ℎ= \alpha 𝐴^𝑇 a$$

The algorithm iterates between updates to $𝑎$ and $ℎ$ until convergence. To ensure convergence, must normalize $𝑎$ and $ℎ$ to have unit Euclidean norm at each iteration. Therefore, the choice of $\alpha$ and $\beta$ are therefore unimportant      

> ** Exercise 03-8:** To understand the HITS algorithm you will now create and test code for this algorithm. Follow these steps:  
> 1. Create a function called `HITS` with arguments of the association matrix, initial hub vector, initial authority vector, and the number of iterations of the algorithm to run. This function does the following:   
>  - Inside a for loop over the number of iterations:  
>    1. Updates the authority vector using the association matrix and the hub vector as argument to the `transition` function. 
>    2. Normalizes the authority vector by using `numpy.divide` with arguments of the updated authority vector and its L2 norm, computed with [numpy.linalg.norm](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html).  
>    3. Updates the hub vector using the association matrix and the authority vector as argument to the `transition` function. 
>    4. Normalizes the hub vector by using `numpy.divide` with arguments of the updated hub vector and its L2 norm, computed with `numpy.linalg.norm`.  
>   - Return the hub and authority vectors
> 2. Initialize an initial hub and authority vector of length the dimension of the association matrix with uniformly distributed values of $\frac{1.0}{dimension(association\ matrix)}$.  
> 3. Execute your function using the association matrix for the 6-page network and the initial hub and authority vectors as arguments. 

In [10]:
def HITS(association, hub, authority, iters=100):
    ## Put your code below
    for _ in range(iters):
        ## Update and normalize the authorithm vector
        authority = transition(association, hub)
        authority = np.divide(authority, np.linalg.norm(authority))
        ## Update and normalize the hubieness 
        hub = transition(np.transpose(association), authority)
        hub = np.divide(hub, np.linalg.norm(hub))
    return hub, authority 

## Compute the intial hub and authority vectors
## Put your code below
n_row = A_deadend.shape[0]
auth_start = np.array([1.0/n_row]*n_row)
hub_start = np.array([1.0/n_row]*n_row)

## Execute your funciton
## Put your code below
HITS(A_deadend, hub_start, auth_start)

(array([0.51721531, 0.5267355 , 0.56154289, 0.28553154, 0.24120396,
        0.        ]),
 array([0.29355748, 0.18692427, 0.18692427, 0.66740602, 0.49650196,
        0.38986875]))

> Examine your results and answer the following questions:
> - Which of the pages have the highest hub scores? Considering the graph of the pages, is this ordering consistent?  
> - Notice the last value of the hub scores. Is this value expected given the graph of the pages? 
> - Which of the pages have the highest authority. Given the in degree of the pages is this ranking consistent?  
> - Compare the ranking of the pages based on authority that found with PageRank. Are these results consistent? 
> **End of exercise.** 

#### Copyright 2021, Stephen F Elston. All rights reserved.